In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter('ignore')

In [3]:
loc = "http://football-data.co.uk/mmz4281/{}/E0.csv"
data_goals = pd.DataFrame()
years = ['1314','1415','1516','1617']
for x in range(1,5):
    r1 = pd.read_csv(loc.format(years[x-1]), error_bad_lines=False, escapechar='\n')
    r1 = r1[['HomeTeam','AwayTeam','FTHG','FTAG','HST','AST']]
    data_goals = data_goals.append(r1)

In [4]:
data_goals = data_goals.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals'})

In [5]:
data_goals.head()

,HomeTeam,AwayTeam,HomeGoals,AwayGoals,HST,AST
0,Arsenal,Aston Villa,1.0,3.0,4.0,4.0
1,Liverpool,Stoke,1.0,0.0,11.0,4.0
2,Norwich,Everton,2.0,2.0,2.0,6.0
3,Sunderland,Fulham,0.0,1.0,3.0,1.0
4,Swansea,Man United,1.0,4.0,6.0,7.0


In [6]:
data_goals.isnull().sum()

HomeTeam     1
AwayTeam     1
HomeGoals    1
AwayGoals    1
HST          1
AST          1
dtype: int64

In [7]:
data_goals = data_goals.dropna()

In [8]:
data_goals.mean()

HomeGoals    1.534211
AwayGoals    1.174342
HST          4.800658
AST          3.826316
dtype: float64

In [9]:
# importing for the Poisson regression model
import statsmodels.api as sm
import statsmodels.formula.api as smf

goal_model_data = pd.concat([data_goals[['HomeTeam','AwayTeam','HomeGoals']].assign(home=1).rename(
            columns={'HomeTeam':'team', 'AwayTeam':'opponent','HomeGoals':'goals','HST':'HomeShots','AST':'AwayShots'}),
           data_goals[['AwayTeam','HomeTeam','AwayGoals']].assign(home=0).rename(
            columns={'AwayTeam':'team', 'HomeTeam':'opponent','AwayGoals':'goals'})])

poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                 3040
Model:                            GLM   Df Residuals:                     2988
Model Family:                 Poisson   Df Model:                           51
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -4360.6
Date:                Tue, 03 Sep 2019   Deviance:                       3422.1
Time:                        05:56:05   Pearson chi2:                 2.96e+03
No. Iterations:                     5   Covariance Type:             nonrobust
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.2099      0.103      2.046      0.041       0.009       0.411
team[T.Aston Villa]           -0.7618      0.118     -6.461      0.000      -0.993      -0.531
team[T.Bournemouth]           -0.2951      0.117     -2.530      0.011      -0.524      -0.066
team[T.Burnley]               -0.7088      0.136     -5.206      0.000      -0.976      -0.442
team[T.Cardiff]               -0.7931      0.187     -4.244      0.000      -1.159      -0.427
team[T.Chelsea]                0.0216      0.084      0.257      0.797      -0.143       0.186
team[T.Crystal Palace]        -0.4954      0.097     -5.083      0.000      -0.686      -0.304
team[T.Everton]               -0.1929      0.089     -2.166      0.030      -0.367      -0.018
team[T.Fulham]                -0.5580      0.169     -3.295      0.001      -0.890      -0.226
team[T.Hull]                  -0.6491      0.113     -5.726      0.000      -0.871      -0.427
team[T.Leicester]             -0.2377      0.099     -2.405      0.016      -0.431      -0.044
team[T.Liverpool]              0.0540      0.084      0.646      0.518      -0.110       0.218
team[T.Man City]               0.1788      0.081      2.208      0.027       0.020       0.338
team[T.Man United]            -0.2085      0.089     -2.339      0.019      -0.383      -0.034
team[T.Middlesbrough]         -0.9143      0.202     -4.533      0.000      -1.310      -0.519
team[T.Newcastle]             -0.4940      0.107     -4.614      0.000      -0.704      -0.284
team[T.Norwich]               -0.7348      0.136     -5.398      0.000      -1.002      -0.468
team[T.QPR]                   -0.4739      0.166     -2.861      0.004      -0.799      -0.149
team[T.Southampton]           -0.2988      0.092     -3.262      0.001      -0.478      -0.119
team[T.Stoke]                 -0.4617      0.096     -4.789      0.000      -0.651      -0.273
team[T.Sunderland]            -0.6138      0.101     -6.049      0.000      -0.813      -0.415
team[T.Swansea]               -0.3909      0.095     -4.136      0.000      -0.576      -0.206
team[T.Tottenham]             -0.0454      0.086     -0.531      0.595      -0.213       0.122
team[T.Watford]               -0.5273      0.127     -4.154      0.000      -0.776      -0.279
team[T.West Brom]             -0.5639      0.100     -5.664      0.000      -0.759      -0.369
team[T.West Ham]              -0.3468      0.093     -3.722      0.000      -0.530      -0.164
opponent[T.Aston Villa]        0.4633      0.107      4.310      0.000       0.253       0.674
opponent[T.Bournemouth]        0.5056      0.118      4.292      0.000       0.275       0.736
opponent[T.Burnley]            0.2784      0.125      2.224      0.026       0.033       0.524
opponent[T.Cardiff]            0.6201      0.141      4.388      0.000       0.343       0.897
opponent[T.Chelsea]        

In [10]:
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=6):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam, 
                                                            'opponent': awayTeam,'home':1},
                                                      index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam, 
                                                            'opponent': homeTeam,'home':0},
                                                      index=[1])).values[0]
    return("HomeTeam Probable goals Scored {}, AwayTeam Probable goals Scored {}".format(home_goals_avg,away_goals_avg))

In [11]:
simulate_match(poisson_model,'Arsenal','Sunderland')

'HomeTeam Probable goals Scored 2.4238993593162226, AwayTeam Probable goals Scored 0.6676936108019813'